## Create test data

Load all needed data

In [1]:
import os
import dipy as dp
import nibabel as nib
import numpy as np
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, reorient_bvecs, GradientTable
from bonndit.shore import ShoreModel, ShoreFit

os.chdir("/home/olivier/Devel/test/samuel/in4michi/")

# Load fractional anisotropy
dti_fa = nib.load("dti_FA.nii.gz")

# Load DTI mask
dti_mask = nib.load("mask.nii.gz")

# Load and adjust tissue segmentation masks
csf_mask = nib.load("fast_pve_0.nii.gz")
gm_mask = nib.load("fast_pve_1.nii.gz")
wm_mask = nib.load("fast_pve_2.nii.gz")

dti_vecs = nib.load("dti_V1.nii.gz")

data = nib.load("data.nii.gz")

bvals, bvecs = read_bvals_bvecs("bvals", "bvecs")
gtab = gradient_table(bvals, bvecs)

/home/olivier/Devel/venvs/bonndit/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def vvi_to_numpy_index(vvi_index):
    new_ind = data.shape[:3] - vvi_index
    return new_ind + (-1,-1,-1)

In [3]:
vvi_to_numpy_index(np.array((95,95,49)))

array([0, 0, 0])

Specify the voxels you want in the test data

In [4]:
wm_voxels = np.array([(49,58,27),(59,33,34),(64,33,32)])
gm_voxels = np.array([(25,44,25),(50,46,25),(55,38,20)])
csf_voxels = np.array([(49,55,25),(38,68,45),(61,28,45)])
add_voxels = np.array([(61,26,23),(60,49,37),(65,24,37)])

TEST_DATA = "/home/olivier/Devel/test/samuel/in4michi/test_data_3d"
if not os.path.exists(TEST_DATA):
    os.mkdir(TEST_DATA)

files = {"dti_FA.nii.gz": dti_fa,
         "mask.nii.gz": dti_mask,
         "fast_pve_0.nii.gz": csf_mask,
         "fast_pve_1.nii.gz": gm_mask,
         "fast_pve_2.nii.gz": wm_mask}

In [5]:
wm_voxels = np.array([tuple(vvi_to_numpy_index(np.array(x))) for x in wm_voxels])

Build the test data

In [6]:
def build_test_data_3d(files):
    for name, img in files.items():
        wm_regions = []
        for voxel in wm_voxels:
            wm_regions.append(np.stack([img.get_data()[tuple(voxel+x)] for x in [(0,0,-1), (0,0,0), (0,0,1)]], axis=0))
        wm_stacked = np.stack(wm_regions, 1)
        
        gm_regions = []
        for voxel in gm_voxels:
            gm_regions.append(np.stack([img.get_data()[tuple(voxel+x)] for x in [(0,0,0), (0,0,0), (0,0,0)]], axis=0))
        gm_stacked = np.stack(gm_regions, 1)
            
        csf_regions = []
        for voxel in csf_voxels:
            csf_regions.append(np.stack([img.get_data()[tuple(voxel+x)] for x in [(0,0,0), (0,0,0), (0,0,0)]], axis=0))
        csf_stacked = np.stack(csf_regions, 1)
        
        complete = np.stack([wm_stacked, gm_stacked, csf_stacked], axis=2)
        print(name, complete.shape)
        img_data = nib.Nifti1Image(complete, img.affine)
        nib.save(img_data, os.path.join(TEST_DATA, name))        

In [7]:
build_test_data_3d(files)
build_test_data_3d({"dti_V1.nii.gz": dti_vecs})
build_test_data_3d({"data.nii.gz": data})

fast_pve_1.nii.gz (3, 3, 3)
fast_pve_0.nii.gz (3, 3, 3)
mask.nii.gz (3, 3, 3)
dti_FA.nii.gz (3, 3, 3)
fast_pve_2.nii.gz (3, 3, 3)
dti_V1.nii.gz (3, 3, 3, 3)
data.nii.gz (3, 3, 3, 97)


## Calculate response result for test data with old code

In [13]:
from subprocess import check_output
from os.path import expanduser
import os

In [14]:
home = expanduser("~")
DATA_DIR = os.path.join(home, 'Devel/test/samuel/in4michi/test_data_3d')
OUT_DIR = os.path.join(DATA_DIR, 'results')

In [15]:
SCRIPT = 'python2 /home/olivier/Devel/mic-tools/scripts/msmt-deconv/shore-response.py '
venv = '. ' + os.path.join(home, 'Devel/venvs/bonndit/bin/activate') + ' && '

command = venv + SCRIPT + '{dire}/ {out}'.format(dire=DATA_DIR, out = OUT_DIR)
print(command)
out = check_output(command, shell = True)

for line in out.splitlines():
    print(line.decode("utf-8") )

. /home/olivier/Devel/venvs/bonndit/bin/activate && python2 /home/olivier/Devel/mic-tools/scripts/msmt-deconv/shore-response.py /home/olivier/Devel/test/samuel/in4michi/test_data_3d/ /home/olivier/Devel/test/samuel/in4michi/test_data_3d/results
('shore path:', '/home/olivier/Devel/mic-tools/packages/dwmri/shore.pyc')
4 4
/home/olivier/Devel/test/samuel/in4michi/test_data_3d/dti_FA.nii.gz
Using provided DTI mask.
loading data....
reshaping...
csf response:
('  0.0%',)
    done   
3 voxel
[  9.16301660e+04  -9.02934493e+04   1.98821362e+05  -2.23875288e+03
   3.44210202e+03   1.16744800e+03  -6.97973007e+03  -1.78303965e+03
   8.46359966e+03  -1.03158867e+04  -3.25269229e+03   1.48611223e+04
   7.66528862e+03   3.40274544e+03  -1.90877332e+03  -7.99447267e+02
   2.13461183e+03  -1.22077248e+02   1.54785180e+03  -1.76300482e+03
   2.37200709e+02  -1.33040030e+03]
[  91630.16601105  -90293.44934797  198821.36167508]
grey matter response:
('  0.0%',)
    done   
5 voxel
[  8.70920574e+04  -

## Calculate deconvolution results for test data with old code

In [12]:
SCRIPT = 'shore-deconvolve.py '
OPTIONS = ['hpsd', 'nonneg', 'none']
venv = '. ' + os.path.join(home, 'Devel/venvs/bonndit/bin/activate') + ' && '

for option in OPTIONS:
    command = venv + SCRIPT + '{dire}/data.nii.gz {dire}/response_complete.npz \
{out}/odf{opt}.nrrd --pos {opt} --csf {out}/csf{opt}.nrrd \
--gm {out}/gm{opt}.nrrd --wm {out}/wm{opt}.nrrd'.format(dire=DATA_DIR, opt=option, out=OUT_DIR)
    print(command)
    out = check_output(command, shell = True)
    for line in out.splitlines():
        print(line.decode("utf-8") )

. /home/olivier/Devel/venvs/bonndit/bin/activate && shore-deconvolve.py /home/olivier/Devel/test/samuel/in4michi/test_data_3d/data.nii.gz /home/olivier/Devel/test/samuel/in4michi/test_data_3d/response_complete.npz /home/olivier/Devel/test/samuel/in4michi/test_data_3d/results/odfhpsd.nrrd --pos hpsd --csf /home/olivier/Devel/test/samuel/in4michi/test_data_3d/results/csfhpsd.nrrd --gm /home/olivier/Devel/test/samuel/in4michi/test_data_3d/results/gmhpsd.nrrd --wm /home/olivier/Devel/test/samuel/in4michi/test_data_3d/results/wmhpsd.nrrd
order (radial / angular): 4 / 4
[[281.77762371   0.           0.         ...   0.         455.33339525
  651.25522943]
 [281.77762371   0.           0.         ...   0.         455.33339525
  651.25522943]
 [281.77762371   0.           0.         ...   0.         455.33339525
  651.25522943]
 ...
 [ 93.2258914  -56.85521783  47.55097727 ...   5.57354002  82.49415443
   29.13357179]
 [ 93.2258914  -16.71205418 -29.03170502 ... -35.39412036  82.49415443
   29